For example, suppose you have the following map:

```
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
```
Visually, the above map of orbits looks like this:

```
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I
```

...


The total number of direct and indirect orbits in this example is `42`.

- B orbits COM: 1
- G orbits B: 1 + 1 = 2
- H orbits G: 1 + 2 = 3
- C orbits B: 1 + 1 = 2
- D orbits C: 1 + 2 = 3
- I orbits D: 1 + 3 = 4
- E orbits I: 1 + 3 = 4
- F orbits E: 1 + 4 = 5
- J orbits E: 1 + 4 = 5
- K orbits J: 1 + 5 = 6 
- L orbits K: 1 + 6 = 7

In [1]:
1 + 2 + 3 + 2 + 3 + 4 + 4 + 5 + 5 + 6 + 7

42

In [22]:
def parse_map(lines):
    object_parents = {}
    
    if isinstance(lines, str):
        lines = lines.split("\n")

    for line in lines:
        line = line.strip()
        if len(line) == 0: continue
            
        # "left)right" = left is parent of right
        left, right = line.split(")")
        assert right not in object_parents
        object_parents[right] = left
    
    return object_parents

m = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L"""

parse_map(m)

{'B': 'COM',
 'C': 'B',
 'D': 'C',
 'E': 'D',
 'F': 'E',
 'G': 'B',
 'H': 'G',
 'I': 'D',
 'J': 'E',
 'K': 'J',
 'L': 'K'}

In [23]:
def checksum(orbit_map):
    memoized_results = {}

    def orbit_count(obj):
        parent = orbit_map[obj]
        if parent == 'COM':
            return 1
        if parent in memoized_results:
            memoized_results[obj] = 1 + memoized_results[parent]
        else:
            memoized_results[obj] = 1 + orbit_count(orbit_map[obj])
        return memoized_results[obj]
    
    counts = [orbit_count(obj) for obj in orbit_map.keys()]
    return sum(counts)

In [24]:
assert 42 == checksum(parse_map(m))

In [49]:
with open('inputs/day6a.txt') as f:
    part1_input = f.readlines()
    
my_input = parse_map(part1_input)

assert 'LH2' in my_input

checksum(my_input)

147223

# part 2

What is the minimum number of orbital transfers required to move from the object YOU are orbiting to the object SAN is orbiting? (Between the objects they are orbiting - not between YOU and SAN.)

In [28]:
m['YOU'], m['SAN']

('PFG', '9FD')

Ideas: 

- find first common ancestor between the two nodes
  - find path from node to root (COM)
  - union the paths, take deepest node
- count number of steps between YOU and ancestor, and same for SAN and ancestor

In [29]:
example = parse_map("""COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN""")

In [30]:
example

{'B': 'COM',
 'C': 'B',
 'D': 'C',
 'E': 'D',
 'F': 'E',
 'G': 'B',
 'H': 'G',
 'I': 'D',
 'J': 'E',
 'K': 'J',
 'L': 'K',
 'YOU': 'K',
 'SAN': 'I'}

In [36]:
def path_to_root(m, node):
    nodes = []
    while node != 'COM':
        nodes.append(node)
        node = m[node]
    nodes.append(node)
    return nodes[::-1]
        
print(path_to_root(example, 'YOU'))

['COM', 'B', 'C', 'D', 'E', 'J', 'K', 'YOU']


In [40]:
def union_paths(p1, p2):
    u = []
    for n in range(min(len(p1), len(p2))):
        if p1[n] == p2[n]:
            u.append(p1[n])
        else:
            break
    return u

assert ['A', 'B', 'C'] == union_paths(['A', 'B', 'C', 'D'], ['A', 'B', 'C', 'E'])
assert ['A', 'B', 'C'] == union_paths(['A', 'B', 'C'], ['A', 'B', 'C', 'E'])


def find_first_common_ancestor(m, node1, node2):
    path1 = path_to_root(m, node1)
    path2 = path_to_root(m, node2)
    u = union_paths(path1, path2)
    return u[-1]

assert 'D' == find_first_common_ancestor(example, 'YOU', 'SAN')

In [46]:
def count_steps(m, node1, node2):
    """Return the number of steps from node2 to node1 in the tree m"""
    count = 0
    n = node2
    while n != node1:
        count += 1
        n = m[n]
    return count

def expect_steps(expected, m, node1, node2):
    actual = count_steps(m, node1, node2)
    assert expected == actual, f'Expected {expected} steps but got {actual}'
    
expect_steps(4, example, 'D', 'YOU')

In [48]:
def minimum_orbital_transfers(m, node1, node2):
    transfer_point = find_first_common_ancestor(m, node1, node2)
    # subtract two because the problem states:
    # "What is the minimum number of orbital transfers required to move from the 
    #  object YOU are orbiting to the object SAN is orbiting? (Between the objects
    #  they are orbiting - not between YOU and SAN.)"
    return count_steps(m, transfer_point, node1) + count_steps(m, transfer_point, node2) - 2

assert 4 == minimum_orbital_transfers(example, 'YOU', 'SAN')
assert 4 == minimum_orbital_transfers(example, 'SAN', 'YOU')

In [50]:
minimum_orbital_transfers(my_input, 'YOU', 'SAN')

340